In [17]:
import os, sys, time
from time import sleep
from datetime import *
from pathlib import Path
import fnmatch, glob, shutil
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import tempfile, logging
from zipfile import ZipFile

In [18]:
from PyPDF2 import PdfFileWriter, PdfFileReader

In [19]:
# SETUP LOGGER
try: #{
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s : %(message)s',
                        datefmt='%Y-%m-%d-%H%M%S',
                        filemode='a')
#}
except: #{
    errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
    errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
    errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    typeE = str("TYPE : " + str(exc_type))
    fileE = str("FILE : " + str(fname))
    lineE = str("LINE : " + str(exc_tb.tb_lineno))
    messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
    print("\n" + typeE +
          "\n" + fileE +
          "\n" + lineE +
          "\n" + messageE)
#}
else: #{
    print("FIN...")
#}

FIN...


In [20]:
# CREATE TIME STAMP (string) FOR ZIP FOLDER
ts = pd.Timestamp.now()
ts_str = str(ts)
ts_str = ts_str[:10]
print(ts_str)

2019-07-29


In [21]:
# DIRECTORIES TO SCAN
in_directory_1 = "F:/APPS/CofA/"
in_directory_2 = "G:/C of A's/#Email Node/"

In [22]:
out_directory = "."

In [23]:
zip_dir_path = os.path.join(out_directory, ts_str)
print(zip_dir_path)

.\2019-07-29


In [24]:
# if 'zip_folder' DOES NOT YET EXIST!
if not os.path.exists(zip_dir_path): #{
    # MAKE IT EXIST!
    os.makedirs(zip_dir_path)
#}

In [25]:
# RESET/RE-ESTABLISH 'out_directory'
out_directory = Path(zip_dir_path)

In [26]:
watermark = "C:/data/inbound/Agilent_CofA_Letterhead_03-21-19.pdf"

## *DEFINE FUNCTIONS*

In [27]:
def create_watermark(input_pdf, output, watermark): #{
    try: #{
        watermark_obj = PdfFileReader(watermark)
        watermark_page = watermark_obj.getPage(0)
        
        pdf_reader = PdfFileReader(input_pdf)
        pdf_writer = PdfFileWriter()
        
        # Watermark all the pages
        for page in range(pdf_reader.getNumPages()): #{
            page = pdf_reader.getPage(page)
            page.mergePage(watermark_page)
            pdf_writer.addPage(page)
        #}
        
        with open(output, 'wb') as out: #{
            pdf_writer.write(out)
        #}
    #}
    except: #{
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.info("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    #}
    else: #{
        logging.info("[Watermark-PDF] FIN...")
    #}
    return
#}

In [28]:
def get_all_file_paths(directory): #{
    
    # initializing empty file paths list
    file_paths = []
    
    # crawling through directory and subdirectories
    for root, directories, files in os.walk(directory): #{
        for filename in files: #{
            # join the two strings in order to form the full filepath
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
        #}
    #}
    
    # returning all file paths
    return file_paths

#}

In [29]:
"""
TAKES IN THE PATH OF A FILE AND DETERMINES:
is modification or created date older? 
returns the more accurate information 
"""
def pull_creation_modification_date(the_file_path): #{
    # GET CREATION DATE
    file_date = os.path.getctime(pdf_path)
    # GET CREATION_DATE
    created = os.stat(pdf_path).st_ctime_ns
    # CONVERT CREATION_DATE TO PANDAS.TIMESTAMP
    c_stamp = pd.Timestamp(created)
    # GET MODIFICATION_TIME
    modified = os.stat(pdf_path).st_mtime_ns
    # CONVERT MODIFIED TIME TO PANDAS.TIMESTAMP
    m_stamp = pd.Timestamp(modified)
    #########################################3##
    # PANDAS.TIMEDELTA FOR SUBTRACTING DATES
    creation_delta = pd.Timedelta(created - modified)
    logging.debug("DELTA TIME (created - modified) >>> " + str(creation_delta))
    # CREATE CHECK DELTA TO USE 'AGAINST' CREATION_DELTA
    check_delta = pd.Timedelta(value=1, unit='min')
    # IF DELTA IS LESS THAN ONE-MINUTE (NEGATIVE)
    if (creation_delta <= check_delta): #{
        logging.debug("<< CREATION DATE IS OLDER >>")
        logging.debug("CREATED == " + str(c_stamp)) # WAS: logging.info
        logging.debug("MODIFIED == " + str(m_stamp))
        # SET STRING VAR FOR EMAIL OUTPUT
        date_str = str(c_stamp)
    #}
    # ELSE... DELTA IS GREATER THAN ONE-MINUTE
    else: #{
        logging.debug("<< MODIFIED DATE IS OLDER >>")
        logging.debug("MODIFIED == " + str(m_stamp)) # WAS: logging.info
        logging.debug("CREATED == " + str(c_stamp))
        # SET STRING VAR FOR EMAIL OUTPUT
        date_str = str(m_stamp)
    #}
    return date_str
#}

In [30]:
in_file = "batch_list.csv"

In [31]:
try: #{
    df_batch = pd.read_csv(in_file, header=None, names=['Lot_No'], engine='python')
#}
except FileNotFoundError: #{
    logging.info("YOU NEED TO CREATE BATCH-LIST FILE !!")
#}
else: #{
    logging.info("[import batch_list] FIN...")
#}

2019-07-29-152053 : [import batch_list] FIN...


In [32]:
df_batch.head(5)

,Lot_No
0,47995N-1
1,DWM-544-1
2,DWM-580-1
3,DWM-583-1
4,DWM-588-1


In [33]:
test_glob = str(in_directory_1 + "*")
print(test_glob)

F:/APPS/CofA/*


In [34]:
# TOTAL counter
x = 0

for lot_no in df_batch.itertuples(): #{
    logging.info("======== " + str(lot_no[1]) + " ========")
    # CHECK AND CREATE NEW DIR PATH
    new_dir_path = os.path.join(out_directory, str(lot_no[1]))
    # IF 'lot_no' DIRECTORY DOES NOT EXIST:
    if not os.path.exists(new_dir_path): #{
        # MAKE IT EXIST!
        os.makedirs(new_dir_path)
    #}
    # CREATE TEMP DIR/FOLDER TO WORK INSIDE OF:
    with tempfile.TemporaryDirectory() as directory_name: #{
        # LOCAL COUNTER
        j = 0
        the_dir = Path(directory_name)
        logging.info("TEMP_DIR== " + str(the_dir))
        # create variable to hold most recent DATETIME
        newest_date = pd.Timestamp.today()
        # create str to hold our most recent file (path)
        newest_cofa = ""
        # GLOB AND ITERATURE THROUGH EACH FILE MATCHING NAME ETC
        for name in sorted(glob.glob(in_directory_1 + "*" + str(lot_no[1]) + "*")): #{
            # CREATE PATH VARIABLE
            pdf_path = Path(str(name))
            ###########################################
            the_date_str = pull_creation_modification_date(pdf_path)
            """
            the_c_date = pd.Timedelta(the_date_str)
            if newest_date > the_c_date: #{
                # THE C_DATE IS OLDER
                newest_date = newest_date
            #}
            else: #{
                # THE "LAST" C_DATE IS OLDER
                # RESET 
                newest_date = the_c_date
            #}
            """
            #####################################################
            logging.info("FILE CREATED == " + the_date_str)
            logging.info("THE_FILE_PATH == " + str(pdf_path))
            logging.info("FILE_EXISTS == " + str(os.path.exists(pdf_path)))
            # GET BASE NAME (FILE NAME)
            file_name = os.path.basename(pdf_path)
            # CREATE TEMP PATH
            temp_path = os.path.join(the_dir, file_name)
            # COPY FILE TO NEW LOCATION
            shutil.copy2(pdf_path, temp_path)
            # CREATE WATERMARK ON NEW FILE
            # (OUTPUT == out_directory + str(lot_no[1]) + filename)
            create_watermark(input_pdf=temp_path,
                            output=os.path.join(new_dir_path, file_name),
                            watermark=watermark)
            # increase counts
            x += 1
            j += 1
            logging.info("LOCAL count == " + str(j))
        #}
    #}
    logging.info("TOTAL count == " + (str(x)))
    logging.info("Directory exists after? " + str(the_dir.exists()))
    logging.info("Contents after: " + str(list(the_dir.glob("*"))))
    logging.info("told you so")
#}

2019-07-29-152053 : ======== 47995N-1 ========
2019-07-29-152053 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmpl01j1vig
2019-07-29-152054 : TOTAL count == 0
2019-07-29-152054 : Directory exists after? False
2019-07-29-152054 : Contents after: []
2019-07-29-152054 : told you so
2019-07-29-152055 : ======== DWM-544-1 ========
2019-07-29-152055 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmps2fkmllc
2019-07-29-152056 : FILE CREATED == 2019-04-17 15:45:44.568289900
2019-07-29-152056 : THE_FILE_PATH == F:\APPS\CofA\DWM-544-1@0006460286.pdf
2019-07-29-152056 : FILE_EXISTS == True
2019-07-29-152056 : [Watermark-PDF] FIN...
2019-07-29-152056 : LOCAL count == 1
2019-07-29-152056 : FILE CREATED == 2019-06-26 20:59:57.088455600
2019-07-29-152056 : THE_FILE_PATH == F:\APPS\CofA\DWM-544-1@0006476293.pdf
2019-07-29-152056 : FILE_EXISTS == True
2019-07-29-152056 : [Watermark-PDF] FIN...
2019-07-29-152056 : LOCAL count == 2
2019-07-29-152056 : TOTAL count == 2
2019-07-29-152056 : Director

2019-07-29-152119 : THE_FILE_PATH == F:\APPS\CofA\IBF-1245@CT-0400.pdf
2019-07-29-152119 : FILE_EXISTS == True
2019-07-29-152119 : [Watermark-PDF] FIN...
2019-07-29-152119 : LOCAL count == 16
2019-07-29-152119 : FILE CREATED == 2019-03-12 12:52:34.773755500
2019-07-29-152119 : THE_FILE_PATH == F:\APPS\CofA\IBF-1245@CT-1199.pdf
2019-07-29-152119 : FILE_EXISTS == True
2019-07-29-152120 : [Watermark-PDF] FIN...
2019-07-29-152120 : LOCAL count == 17
2019-07-29-152120 : TOTAL count == 22
2019-07-29-152120 : Directory exists after? False
2019-07-29-152120 : Contents after: []
2019-07-29-152120 : told you so
2019-07-29-152120 : ======== ICP-042 ========
2019-07-29-152120 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmp3ja6y9i1
2019-07-29-152122 : FILE CREATED == 2015-03-17 15:28:52.008451600
2019-07-29-152122 : THE_FILE_PATH == F:\APPS\CofA\ICP-042@CL-4904.pdf
2019-07-29-152122 : FILE_EXISTS == True
2019-07-29-152122 : [Watermark-PDF] FIN...
2019-07-29-152122 : LOCAL count == 1
2019-07-2

2019-07-29-152142 : TOTAL count == 49
2019-07-29-152142 : Directory exists after? False
2019-07-29-152142 : Contents after: []
2019-07-29-152142 : told you so
2019-07-29-152142 : ======== ICP-125 ========
2019-07-29-152142 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmpktqv7vnv
2019-07-29-152144 : FILE CREATED == 2019-06-03 18:13:15.118574100
2019-07-29-152144 : THE_FILE_PATH == F:\APPS\CofA\ICP-125@0006470502.pdf
2019-07-29-152144 : FILE_EXISTS == True
2019-07-29-152145 : [Watermark-PDF] FIN...
2019-07-29-152145 : LOCAL count == 1
2019-07-29-152145 : FILE CREATED == 2015-05-12 10:40:07.798819800
2019-07-29-152145 : THE_FILE_PATH == F:\APPS\CofA\ICP-125@CM-2432.pdf
2019-07-29-152145 : FILE_EXISTS == True
2019-07-29-152145 : [Watermark-PDF] FIN...
2019-07-29-152145 : LOCAL count == 2
2019-07-29-152145 : FILE CREATED == 2018-04-09 16:23:45.626666300
2019-07-29-152145 : THE_FILE_PATH == F:\APPS\CofA\ICP-125@CS-1643.pdf
2019-07-29-152145 : FILE_EXISTS == True
2019-07-29-152146 : [Wat

2019-07-29-152205 : [Watermark-PDF] FIN...
2019-07-29-152205 : LOCAL count == 2
2019-07-29-152205 : FILE CREATED == 2018-04-05 16:07:52.054406500
2019-07-29-152205 : THE_FILE_PATH == F:\APPS\CofA\ICP-333@AU-01246.pdf
2019-07-29-152205 : FILE_EXISTS == True
2019-07-29-152206 : [Watermark-PDF] FIN...
2019-07-29-152206 : LOCAL count == 3
2019-07-29-152206 : TOTAL count == 79
2019-07-29-152206 : Directory exists after? False
2019-07-29-152206 : Contents after: []
2019-07-29-152206 : told you so
2019-07-29-152206 : ======== ICP-356 ========
2019-07-29-152206 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmp36ywygdi
2019-07-29-152207 : FILE CREATED == 2015-03-30 11:47:51.073000
2019-07-29-152207 : THE_FILE_PATH == F:\APPS\CofA\ICP-356@AU-00033.pdf
2019-07-29-152207 : FILE_EXISTS == True
2019-07-29-152208 : [Watermark-PDF] FIN...
2019-07-29-152208 : LOCAL count == 1
2019-07-29-152208 : FILE CREATED == 2015-08-25 14:20:14.326384
2019-07-29-152208 : THE_FILE_PATH == F:\APPS\CofA\ICP-356@AU-

2019-07-29-152216 : FILE CREATED == 2017-02-03 16:54:50.318023400
2019-07-29-152216 : THE_FILE_PATH == F:\APPS\CofA\IMS-102@CR-0285.pdf
2019-07-29-152216 : FILE_EXISTS == True
2019-07-29-152217 : [Watermark-PDF] FIN...
2019-07-29-152217 : LOCAL count == 4
2019-07-29-152217 : FILE CREATED == 2018-03-27 22:04:08.765326100
2019-07-29-152217 : THE_FILE_PATH == F:\APPS\CofA\IMS-102@CS-1404.pdf
2019-07-29-152217 : FILE_EXISTS == True
2019-07-29-152218 : [Watermark-PDF] FIN...
2019-07-29-152218 : LOCAL count == 5
2019-07-29-152218 : FILE CREATED == 2015-05-14 17:32:17.472261300
2019-07-29-152218 : THE_FILE_PATH == F:\APPS\CofA\IMS-102@M00891.pdf
2019-07-29-152218 : FILE_EXISTS == True
2019-07-29-152218 : [Watermark-PDF] FIN...
2019-07-29-152218 : LOCAL count == 6
2019-07-29-152218 : FILE CREATED == 2015-05-14 17:33:23.191431900
2019-07-29-152218 : THE_FILE_PATH == F:\APPS\CofA\IMS-102@P00833.pdf
2019-07-29-152218 : FILE_EXISTS == True
2019-07-29-152218 : [Watermark-PDF] FIN...
2019-07-29-1522

...

2019-07-29-152536 : LOCAL count == 3
2019-07-29-152536 : FILE CREATED == 2014-10-06 17:50:36.449026100
2019-07-29-152536 : THE_FILE_PATH == F:\APPS\CofA\RPC-164AS@CL-3988.PDF
2019-07-29-152536 : FILE_EXISTS == True
2019-07-29-152536 : [Watermark-PDF] FIN...
2019-07-29-152536 : LOCAL count == 4
2019-07-29-152536 : FILE CREATED == 2016-10-27 12:54:57.347315700
2019-07-29-152536 : THE_FILE_PATH == F:\APPS\CofA\RPC-164AS@CP-5366.PDF
2019-07-29-152536 : FILE_EXISTS == True
2019-07-29-152537 : [Watermark-PDF] FIN...
2019-07-29-152537 : LOCAL count == 5
2019-07-29-152537 : FILE CREATED == 2018-11-07 13:10:58.408492800
2019-07-29-152537 : THE_FILE_PATH == F:\APPS\CofA\RPC-164AS@CS-5381.PDF
2019-07-29-152537 : FILE_EXISTS == True
2019-07-29-152537 : [Watermark-PDF] FIN...
2019-07-29-152537 : LOCAL count == 6
2019-07-29-152537 : TOTAL count == 426
2019-07-29-152537 : Directory exists after? False
2019-07-29-152537 : Contents after: []
2019-07-29-152537 : told you so
2019-07-29-152537 : ======== 

2019-07-29-152608 : TOTAL count == 436
2019-07-29-152608 : Directory exists after? False
2019-07-29-152608 : Contents after: []
2019-07-29-152608 : told you so
2019-07-29-152608 : ======== UST-200-1 ========
2019-07-29-152608 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmp7_3be7rm
2019-07-29-152610 : TOTAL count == 436
2019-07-29-152610 : Directory exists after? False
2019-07-29-152610 : Contents after: []
2019-07-29-152610 : told you so
2019-07-29-152610 : ======== WRK-151 ========
2019-07-29-152610 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmpgjz22my5
2019-07-29-152612 : FILE CREATED == 2016-03-07 20:10:07.380101900
2019-07-29-152612 : THE_FILE_PATH == F:\APPS\CofA\WRK-151A@NT053304.pdf
2019-07-29-152612 : FILE_EXISTS == True
2019-07-29-152612 : [Watermark-PDF] FIN...
2019-07-29-152612 : LOCAL count == 1
2019-07-29-152612 : FILE CREATED == 2017-06-06 13:23:52.816825600
2019-07-29-152612 : THE_FILE_PATH == F:\APPS\CofA\WRK-151A@NT057373.pdf
2019-07-29-152612 : FILE_EXIST

2019-07-29-152622 : FILE_EXISTS == True
2019-07-29-152623 : [Watermark-PDF] FIN...
2019-07-29-152623 : LOCAL count == 30
2019-07-29-152623 : FILE CREATED == 2017-06-05 15:43:22.419356500
2019-07-29-152623 : THE_FILE_PATH == F:\APPS\CofA\WRK-151O@NT057361.pdf
2019-07-29-152623 : FILE_EXISTS == True
2019-07-29-152623 : [Watermark-PDF] FIN...
2019-07-29-152623 : LOCAL count == 31
2019-07-29-152623 : TOTAL count == 467
2019-07-29-152623 : Directory exists after? False
2019-07-29-152623 : Contents after: []
2019-07-29-152623 : told you so
2019-07-29-152623 : ======== WRK-151A ========
2019-07-29-152623 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmpsqqu55l8
2019-07-29-152625 : FILE CREATED == 2016-03-07 20:10:07.380101900
2019-07-29-152625 : THE_FILE_PATH == F:\APPS\CofA\WRK-151A@NT053304.pdf
2019-07-29-152625 : FILE_EXISTS == True
2019-07-29-152625 : [Watermark-PDF] FIN...
2019-07-29-152625 : LOCAL count == 1
2019-07-29-152625 : FILE CREATED == 2017-06-06 13:23:52.816825600
2019-07-2

2019-07-29-152645 : THE_FILE_PATH == F:\APPS\CofA\WRK-151J@NT057356.pdf
2019-07-29-152645 : FILE_EXISTS == True
2019-07-29-152645 : [Watermark-PDF] FIN...
2019-07-29-152645 : LOCAL count == 2
2019-07-29-152645 : TOTAL count == 487
2019-07-29-152645 : Directory exists after? False
2019-07-29-152645 : Contents after: []
2019-07-29-152645 : told you so
2019-07-29-152645 : ======== WRK-151K ========
2019-07-29-152645 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmp54oqflcb
2019-07-29-152647 : FILE CREATED == 2016-03-07 21:32:45.179849100
2019-07-29-152647 : THE_FILE_PATH == F:\APPS\CofA\WRK-151K@NT053313.pdf
2019-07-29-152647 : FILE_EXISTS == True
2019-07-29-152647 : [Watermark-PDF] FIN...
2019-07-29-152647 : LOCAL count == 1
2019-07-29-152647 : FILE CREATED == 2017-06-05 15:27:45.510743200
2019-07-29-152647 : THE_FILE_PATH == F:\APPS\CofA\WRK-151K@NT057357.pdf
2019-07-29-152647 : FILE_EXISTS == True
2019-07-29-152647 : [Watermark-PDF] FIN...
2019-07-29-152647 : LOCAL count == 2
2019-

2019-07-29-152715 : TOTAL count == 498
2019-07-29-152715 : Directory exists after? False
2019-07-29-152715 : Contents after: []
2019-07-29-152715 : told you so
2019-07-29-152715 : ======== PP-301-1 ========
2019-07-29-152715 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmpnta5fbhv
2019-07-29-152717 : TOTAL count == 498
2019-07-29-152717 : Directory exists after? False
2019-07-29-152717 : Contents after: []
2019-07-29-152717 : told you so
2019-07-29-152717 : ======== PP-311-1 ========
2019-07-29-152717 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmpkd2s14pj
2019-07-29-152718 : TOTAL count == 498
2019-07-29-152718 : Directory exists after? False
2019-07-29-152718 : Contents after: []
2019-07-29-152718 : told you so
2019-07-29-152718 : ======== PP-341-1 ========
2019-07-29-152718 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmpa4paiev9
2019-07-29-152720 : TOTAL count == 498
2019-07-29-152720 : Directory exists after? False
2019-07-29-152720 : Contents after: []
2019-07-29-

## *`os.walk()` thru WORKING DIRECTORY* 

## and keep only **MOST RECENT** file 

## `or`

## keep only **MOST OLDEST** file

In [35]:
most_recent = False

In [36]:
type(most_recent)

bool

In [37]:
if most_recent is True: #{
    print("true")
#}
else: #{
    print("false!")
#}

false!


In [38]:
in_directory = "."

In [39]:
# CREATE LIST VARIABLES THAT WILL BECOME COLS IN DATAFRAME
file_col = []
date_col = []

---
*PROCESS FOR RETURNING `MOST RECENT`*

In [40]:
# create LIST vars to hold all the most recent files/dates
recent_files = []
recent_dates = []

In [41]:
# iterate thru directory/directories 
for root, d_names, f_names in os.walk(in_directory): #{
    logging.info("===================== START ====================== \n")
    logging.info("\t *root* = " + str(root) + "\n") 
    logging.info("\t *dict* = " + str(d_names) + "\n")
    if len(f_names) >= 0: #{
        logging.info("\t *file(s)*: \n")
        # CREATE VARS for "most_recents"
        most_recent_date = pd.Timestamp(year=1999, month=12, day=31)
        most_recent_name = "NaN"
        # multiple file(s)
        for f in f_names: #{
            logging.info("\t FILE-NAME == " + str(f) + "\n")
            file_name = os.path.basename(f)
            # add **file NAME** to list
            file_col.append(file_name)
            # create variable for *ORIGINAL* file_path
            f_path = os.path.join("F:/APPS/CofA/", file_name)
            logging.info("\t F_PATH == " + str(f_path) + "\n")
            if os.path.exists(f_path): #{
                # create date var
                date = os.path.getctime(f_path)
                # FORMAT date var & SET AS str
                date_str = str(datetime.fromtimestamp(date))
                # set our date STR to a Pandas.TIMESTAMP var
                ts = pd.Timestamp(ts_input=date_str)
                logging.info("\t CREATE_DATE == " + str(ts) + "\n")
                ##################################################### 
                # CHECK IF MOST RECENT (if 'ts' is more recent...)
                if ts > most_recent_date: #{
                    # RE_ASSIGN 'most_recent_date'
                    most_recent_date = ts
                    most_recent_name = f_path
                    logging.info("\t most_recent_date == " + str(most_recent_date) + "\n")
                    logging.info("\t most_recent_name == " + str(most_recent_name) + "\n")
                #}
                else: #{
                    # keep "most_recent_date" the same
                    logging.info("\t most_recent_date == " + str(most_recent_date) + "\n")
                    logging.info("\t most_recent_name == " + str(most_recent_name) + "\n")
                #}
                #logging.info("\t ts == " + str(ts) + "\n")
                # add **file DATE** to list
                date_col.append(date_str)
            #}
            else: #{
                logging.info("\t CREATE_DATE == NaN (doesn't exist) \n")
            #}
        #}
    #}
    else: #{
        # no files
        logging.info("\t NO-FILES!")
    #}
    logging.info("=====================  END  ======================")
    recent_files.append(most_recent_name)
    recent_dates.append(most_recent_date)
#}

2019-07-29-152759 : ===================== START ====================== 

2019-07-29-152759 : 	 *root* = .

2019-07-29-152759 : 	 *dict* = ['.ipynb_checkpoints', '2019-07-29']

2019-07-29-152759 : 	 *file(s)*: 

2019-07-29-152759 : 	 FILE-NAME == batch_list.csv

2019-07-29-152759 : 	 F_PATH == F:/APPS/CofA/batch_list.csv

2019-07-29-152759 : 	 CREATE_DATE == NaN (doesn't exist) 

2019-07-29-152759 : 	 FILE-NAME == CC_2019_07_29.ipynb

2019-07-29-152759 : 	 F_PATH == F:/APPS/CofA/CC_2019_07_29.ipynb

2019-07-29-152759 : 	 CREATE_DATE == NaN (doesn't exist) 

2019-07-29-152759 : 	 FILE-NAME == string_formatting_testing.ipynb

2019-07-29-152759 : 	 F_PATH == F:/APPS/CofA/string_formatting_testing.ipynb

2019-07-29-152759 : 	 CREATE_DATE == NaN (doesn't exist) 

2019-07-29-152759 : =====================  END  ======================
2019-07-29-152759 : ===================== START ====================== 

2019-07-29-152759 : 	 *root* = .\.ipynb_checkpoints

2019-07-29-152759 : 	 *dict* = []



2019-07-29-152759 : 	 FILE-NAME == IBF-1245@CL-3705.pdf

2019-07-29-152759 : 	 F_PATH == F:/APPS/CofA/IBF-1245@CL-3705.pdf

2019-07-29-152759 : 	 CREATE_DATE == 2014-09-24 12:22:58.465746

2019-07-29-152759 : 	 most_recent_date == 2015-03-31 09:06:37.919280

2019-07-29-152759 : 	 most_recent_name == F:/APPS/CofA/IBF-1245@CL-1912.pdf

2019-07-29-152759 : 	 FILE-NAME == IBF-1245@CM-2198.pdf

2019-07-29-152759 : 	 F_PATH == F:/APPS/CofA/IBF-1245@CM-2198.pdf

2019-07-29-152759 : 	 CREATE_DATE == 2015-05-04 08:42:37.966016

2019-07-29-152759 : 	 most_recent_date == 2015-05-04 08:42:37.966016

2019-07-29-152759 : 	 most_recent_name == F:/APPS/CofA/IBF-1245@CM-2198.pdf

2019-07-29-152759 : 	 FILE-NAME == IBF-1245@CM-4282.pdf

2019-07-29-152759 : 	 F_PATH == F:/APPS/CofA/IBF-1245@CM-4282.pdf

2019-07-29-152759 : 	 CREATE_DATE == 2015-08-11 14:47:54.769735

2019-07-29-152759 : 	 most_recent_date == 2015-08-11 14:47:54.769735

2019-07-29-152759 : 	 most_recent_name == F:/APPS/CofA/IBF-1245@CM-42

2019-07-29-152759 : 	 most_recent_name == F:/APPS/CofA/ICP-042@CS-0671.pdf

2019-07-29-152759 : =====================  END  ======================
2019-07-29-152759 : ===================== START ====================== 

2019-07-29-152759 : 	 *root* = .\2019-07-29\ICP-048

2019-07-29-152759 : 	 *dict* = []

2019-07-29-152759 : 	 *file(s)*: 

2019-07-29-152759 : 	 FILE-NAME == ICP-048@CL-2608.pdf

2019-07-29-152759 : 	 F_PATH == F:/APPS/CofA/ICP-048@CL-2608.pdf

2019-07-29-152759 : 	 CREATE_DATE == 2015-03-16 12:17:16.714212

2019-07-29-152759 : 	 most_recent_date == 2015-03-16 12:17:16.714212

2019-07-29-152759 : 	 most_recent_name == F:/APPS/CofA/ICP-048@CL-2608.pdf

2019-07-29-152759 : 	 FILE-NAME == ICP-048@CP-5285.pdf

2019-07-29-152759 : 	 F_PATH == F:/APPS/CofA/ICP-048@CP-5285.pdf

2019-07-29-152759 : 	 CREATE_DATE == 2016-10-25 06:50:05.473080

2019-07-29-152759 : 	 most_recent_date == 2016-10-25 06:50:05.473080

2019-07-29-152759 : 	 most_recent_name == F:/APPS/CofA/ICP-048@CP-5

2019-07-29-152800 : 	 FILE-NAME == ICP-125@G00793A.pdf

2019-07-29-152800 : 	 F_PATH == F:/APPS/CofA/ICP-125@G00793A.pdf

2019-07-29-152800 : 	 CREATE_DATE == 2015-04-23 14:01:29.922401

2019-07-29-152800 : 	 most_recent_date == 2019-06-03 14:20:36.381002

2019-07-29-152800 : 	 most_recent_name == F:/APPS/CofA/ICP-125@0006470502.pdf

2019-07-29-152800 : 	 FILE-NAME == ICP-125@IH-0391.pdf

2019-07-29-152800 : 	 F_PATH == F:/APPS/CofA/ICP-125@IH-0391.pdf

2019-07-29-152800 : 	 CREATE_DATE == 2015-11-05 09:50:32.856187

2019-07-29-152800 : 	 most_recent_date == 2019-06-03 14:20:36.381002

2019-07-29-152800 : 	 most_recent_name == F:/APPS/CofA/ICP-125@0006470502.pdf

2019-07-29-152800 : 	 FILE-NAME == ICP-125@J00797.pdf

2019-07-29-152800 : 	 F_PATH == F:/APPS/CofA/ICP-125@J00797.pdf

2019-07-29-152800 : 	 CREATE_DATE == 2015-04-23 14:18:52.813026

2019-07-29-152800 : 	 most_recent_date == 2019-06-03 14:20:36.381002

2019-07-29-152800 : 	 most_recent_name == F:/APPS/CofA/ICP-125@0006470502

2019-07-29-152800 : 	 most_recent_name == F:/APPS/CofA/ICP-126@0006465163.pdf

2019-07-29-152800 : =====================  END  ======================
2019-07-29-152800 : ===================== START ====================== 

2019-07-29-152800 : 	 *root* = .\2019-07-29\ICP-333

2019-07-29-152800 : 	 *dict* = []

2019-07-29-152800 : 	 *file(s)*: 

2019-07-29-152800 : 	 FILE-NAME == ICP-333@AU-00025.pdf

2019-07-29-152800 : 	 F_PATH == F:/APPS/CofA/ICP-333@AU-00025.pdf

2019-07-29-152800 : 	 CREATE_DATE == 2015-04-17 09:58:40.036000

2019-07-29-152800 : 	 most_recent_date == 2015-04-17 09:58:40.036000

2019-07-29-152800 : 	 most_recent_name == F:/APPS/CofA/ICP-333@AU-00025.pdf

2019-07-29-152800 : 	 FILE-NAME == ICP-333@AU-00938.pdf

2019-07-29-152800 : 	 F_PATH == F:/APPS/CofA/ICP-333@AU-00938.pdf

2019-07-29-152800 : 	 CREATE_DATE == 2015-11-06 15:04:48.524902

2019-07-29-152800 : 	 most_recent_date == 2015-11-06 15:04:48.524902

2019-07-29-152800 : 	 most_recent_name == F:/APPS/CofA/ICP-

...

2019-07-29-152806 : 	 FILE-NAME == RPC-164AS@CP-5366.PDF

2019-07-29-152806 : 	 F_PATH == F:/APPS/CofA/RPC-164AS@CP-5366.PDF

2019-07-29-152806 : 	 CREATE_DATE == 2016-10-27 08:54:57.347316

2019-07-29-152806 : 	 most_recent_date == 2016-10-27 08:54:57.347316

2019-07-29-152806 : 	 most_recent_name == F:/APPS/CofA/RPC-164AS@CP-5366.PDF

2019-07-29-152806 : 	 FILE-NAME == RPC-164AS@CS-5381.PDF

2019-07-29-152806 : 	 F_PATH == F:/APPS/CofA/RPC-164AS@CS-5381.PDF

2019-07-29-152806 : 	 CREATE_DATE == 2018-11-07 08:10:58.408493

2019-07-29-152806 : 	 most_recent_date == 2018-11-07 08:10:58.408493

2019-07-29-152806 : 	 most_recent_name == F:/APPS/CofA/RPC-164AS@CS-5381.PDF

2019-07-29-152806 : =====================  END  ======================
2019-07-29-152806 : ===================== START ====================== 

2019-07-29-152806 : 	 *root* = .\2019-07-29\RPC-164S

2019-07-29-152806 : 	 *dict* = []

2019-07-29-152806 : 	 *file(s)*: 

2019-07-29-152806 : 	 FILE-NAME == RPC-164S@CE-3735.pd

2019-07-29-152806 : 	 *root* = .\2019-07-29\UST-200-1

2019-07-29-152806 : 	 *dict* = []

2019-07-29-152806 : 	 *file(s)*: 

2019-07-29-152806 : =====================  END  ======================
2019-07-29-152806 : ===================== START ====================== 

2019-07-29-152806 : 	 *root* = .\2019-07-29\WRK-151

2019-07-29-152806 : 	 *dict* = []

2019-07-29-152806 : 	 *file(s)*: 

2019-07-29-152806 : 	 FILE-NAME == WRK-151A@NT053304.pdf

2019-07-29-152806 : 	 F_PATH == F:/APPS/CofA/WRK-151A@NT053304.pdf

2019-07-29-152806 : 	 CREATE_DATE == 2019-05-28 10:03:49.204060

2019-07-29-152806 : 	 most_recent_date == 2019-05-28 10:03:49.204060

2019-07-29-152806 : 	 most_recent_name == F:/APPS/CofA/WRK-151A@NT053304.pdf

2019-07-29-152806 : 	 FILE-NAME == WRK-151A@NT057373.pdf

2019-07-29-152806 : 	 F_PATH == F:/APPS/CofA/WRK-151A@NT057373.pdf

2019-07-29-152806 : 	 CREATE_DATE == 2019-05-28 09:54:42.087682

2019-07-29-152806 : 	 most_recent_date == 2019-05-28 10:03:49.204060

2019-07-

2019-07-29-152807 : 	 F_PATH == F:/APPS/CofA/WRK-151L@NT057358.pdf

2019-07-29-152807 : 	 CREATE_DATE == 2019-05-28 09:54:43.040819

2019-07-29-152807 : 	 most_recent_date == 2019-05-28 10:03:49.204060

2019-07-29-152807 : 	 most_recent_name == F:/APPS/CofA/WRK-151A@NT053304.pdf

2019-07-29-152807 : 	 FILE-NAME == WRK-151M@0006464096.pdf

2019-07-29-152807 : 	 F_PATH == F:/APPS/CofA/WRK-151M@0006464096.pdf

2019-07-29-152807 : 	 CREATE_DATE == 2019-05-18 11:49:49.181832

2019-07-29-152807 : 	 most_recent_date == 2019-05-28 10:03:49.204060

2019-07-29-152807 : 	 most_recent_name == F:/APPS/CofA/WRK-151A@NT053304.pdf

2019-07-29-152807 : 	 FILE-NAME == WRK-151M@NT053318.pdf

2019-07-29-152807 : 	 F_PATH == F:/APPS/CofA/WRK-151M@NT053318.pdf

2019-07-29-152807 : 	 CREATE_DATE == 2019-05-28 10:03:47.547789

2019-07-29-152807 : 	 most_recent_date == 2019-05-28 10:03:49.204060

2019-07-29-152807 : 	 most_recent_name == F:/APPS/CofA/WRK-151A@NT053304.pdf

2019-07-29-152807 : 	 FILE-NAME == WR

2019-07-29-152807 : 	 most_recent_name == F:/APPS/CofA/WRK-151F@NT053322.pdf

2019-07-29-152807 : =====================  END  ======================
2019-07-29-152807 : ===================== START ====================== 

2019-07-29-152807 : 	 *root* = .\2019-07-29\WRK-151G

2019-07-29-152807 : 	 *dict* = []

2019-07-29-152807 : 	 *file(s)*: 

2019-07-29-152807 : 	 FILE-NAME == WRK-151G@NT053310.pdf

2019-07-29-152807 : 	 F_PATH == F:/APPS/CofA/WRK-151G@NT053310.pdf

2019-07-29-152807 : 	 CREATE_DATE == 2019-05-28 10:03:48.532177

2019-07-29-152807 : 	 most_recent_date == 2019-05-28 10:03:48.532177

2019-07-29-152807 : 	 most_recent_name == F:/APPS/CofA/WRK-151G@NT053310.pdf

2019-07-29-152807 : 	 FILE-NAME == WRK-151G@NT057353.pdf

2019-07-29-152807 : 	 F_PATH == F:/APPS/CofA/WRK-151G@NT057353.pdf

2019-07-29-152807 : 	 CREATE_DATE == 2019-05-28 09:54:43.634577

2019-07-29-152807 : 	 most_recent_date == 2019-05-28 10:03:48.532177

2019-07-29-152807 : 	 most_recent_name == F:/APPS/CofA

2019-07-29-152807 : 	 *file(s)*: 

2019-07-29-152807 : 	 FILE-NAME == WRK-151O@NT053315.pdf

2019-07-29-152807 : 	 F_PATH == F:/APPS/CofA/WRK-151O@NT053315.pdf

2019-07-29-152807 : 	 CREATE_DATE == 2019-05-28 10:03:47.938419

2019-07-29-152807 : 	 most_recent_date == 2019-05-28 10:03:47.938419

2019-07-29-152807 : 	 most_recent_name == F:/APPS/CofA/WRK-151O@NT053315.pdf

2019-07-29-152807 : 	 FILE-NAME == WRK-151O@NT057361.pdf

2019-07-29-152807 : 	 F_PATH == F:/APPS/CofA/WRK-151O@NT057361.pdf

2019-07-29-152807 : 	 CREATE_DATE == 2019-05-28 09:54:42.775191

2019-07-29-152807 : 	 most_recent_date == 2019-05-28 10:03:47.938419

2019-07-29-152807 : 	 most_recent_name == F:/APPS/CofA/WRK-151O@NT053315.pdf

2019-07-29-152807 : =====================  END  ======================
2019-07-29-152807 : ===================== START ====================== 

2019-07-29-152807 : 	 *root* = .\2019-07-29\WRK-151P

2019-07-29-152807 : 	 *dict* = []

2019-07-29-152807 : 	 *file(s)*: 

2019-07-29-152807 :

In [42]:
# CREATE EMPTY DATAFRAME
df_dates = pd.DataFrame()

In [43]:
# CREATE SERIES OFF LIST
f1 = pd.Series(recent_files)

In [44]:
# CREATE SERIES OFF LIST
d1 = pd.Series(recent_dates)

In [45]:
df_dates['File'] = f1
df_dates['Time'] = d1

In [46]:
df_dates.head(10)

,File,Time
0,NaN,1999-12-31 00:00:00.000000
1,NaN,1999-12-31 00:00:00.000000
2,NaN,1999-12-31 00:00:00.000000
3,NaN,1999-12-31 00:00:00.000000
4,F:/APPS/CofA/DWM-544-1@0006476293.pdf,2019-06-26 16:59:57.088456
5,F:/APPS/CofA/DWM-580-1@0006472371.pdf,2019-06-10 06:26:02.682714
6,F:/APPS/CofA/DWM-583-1@CG-3973.PDF,2010-12-21 14:30:59.128316
7,NaN,1999-12-31 00:00:00.000000
8,NaN,1999-12-31 00:00:00.000000
9,NaN,1999-12-31 00:00:00.000000


### *REPLACE `NaN` VALUES with REAL IN DATAFRAME*

In [61]:
df_dates.replace(regex=['NaN'], value=np.nan, inplace=True)

## *DROP `NA` VALUES FROM DATAFRAME*

In [62]:
df_dates.dropna(how='any', inplace=True)

In [63]:
df_dates.head(10)

,File,Time
4,F:/APPS/CofA/DWM-544-1@0006476293.pdf,2019-06-26 16:59:57.088456
5,F:/APPS/CofA/DWM-580-1@0006472371.pdf,2019-06-10 06:26:02.682714
6,F:/APPS/CofA/DWM-583-1@CG-3973.PDF,2010-12-21 14:30:59.128316
13,F:/APPS/CofA/IBF-1245@CT-1199.pdf,2019-03-12 09:34:33.149765
14,F:/APPS/CofA/ICP-042@CS-0671.pdf,2018-02-15 08:55:35.820588
15,F:/APPS/CofA/ICP-048@CP-5285.pdf,2016-10-25 06:50:05.473080
16,F:/APPS/CofA/ICP-049@CS-6387.pdf,2019-01-28 16:07:49.879841
17,F:/APPS/CofA/ICP-125@0006470502.pdf,2019-06-03 14:20:36.381002
18,F:/APPS/CofA/ICP-126@0006465163.pdf,2019-05-13 13:20:41.915678
19,F:/APPS/CofA/ICP-333@AU-01246.pdf,2018-04-05 12:11:16.278399


In [64]:
len(df_dates)

77

### *CREATE DIRECTORY FOR `MOST_RECENT`*

In [71]:
new_dir_path = os.path.join(out_directory, "_most_recent")
print(new_dir_path)

2019-07-29\_most_recent


In [72]:
if not os.path.exists(new_dir_path): #{
    # MAKE IT EXIST!
    os.makedirs(new_dir_path)
#}

## *itertupples()* to iterate through `DataFrame` 

## *CREATE ZIPS* in `temp_dir` (*for ALL copies*) but `only` for recent

In [74]:
# CREATE TEMP DIR/FOLDER TO WORK INSIDE OF:
with tempfile.TemporaryDirectory() as directory_name: #{
    # LOCAL COUNTER
    j = 0
    the_dir = Path(directory_name)
    logging.info("TEMP_DIR== " + str(the_dir))
    for most_recent in df_dates.itertuples(index=False, name='Most_Recent'): #{
        # CREATE PATH VARIABLE
        pdf_path = Path(str(most_recent[0]))
        # GET BASE NAME (FILE NAME)
        file_name = os.path.basename(pdf_path)
        # CREATE TEMP PATH
        temp_path = os.path.join(the_dir, file_name)
        # COPY FILE TO NEW (TEMP) LOCATION
        shutil.copy2(pdf_path, temp_path)
        # CREATE WATERMARK ON NEW FILE
        create_watermark(input_pdf=temp_path,
                        output=os.path.join(new_dir_path, file_name),
                        watermark=watermark)
        # INCREASE COUNT
        j += 1
    #}
#}
logging.info("MOST_RECENT_COUNT == " + str(j))
logging.info("Directory exists after? " + str(the_dir.exists()))
logging.info("Contents after: " + str(list(the_dir.glob("*"))))

2019-07-30-114508 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmp8csouhc1
2019-07-30-114508 : [Watermark-PDF] FIN...
2019-07-30-114509 : [Watermark-PDF] FIN...
2019-07-30-114509 : [Watermark-PDF] FIN...
2019-07-30-114510 : [Watermark-PDF] FIN...
2019-07-30-114511 : [Watermark-PDF] FIN...
2019-07-30-114511 : [Watermark-PDF] FIN...
2019-07-30-114512 : [Watermark-PDF] FIN...
2019-07-30-114514 : [Watermark-PDF] FIN...
2019-07-30-114514 : [Watermark-PDF] FIN...
2019-07-30-114515 : [Watermark-PDF] FIN...
2019-07-30-114516 : [Watermark-PDF] FIN...
2019-07-30-114516 : [Watermark-PDF] FIN...
2019-07-30-114517 : [Watermark-PDF] FIN...
2019-07-30-114517 : [Watermark-PDF] FIN...
2019-07-30-114518 : [Watermark-PDF] FIN...
2019-07-30-114519 : [Watermark-PDF] FIN...
2019-07-30-114519 : [Watermark-PDF] FIN...
2019-07-30-114519 : [Watermark-PDF] FIN...
2019-07-30-114519 : [Watermark-PDF] FIN...
2019-07-30-114519 : [Watermark-PDF] FIN...
2019-07-30-114519 : [Watermark-PDF] FIN...
2019-07-30-114520